In [1]:
# ==============================================================================
# AURA AI ENGINE - ADIM 3: VERİ ÖN İŞLEME VE TEMİZLEME
# ==============================================================================
# AMAÇ: Keşifsel veri analizi sırasında tespit edilen sorunları (dengesizlik,
#       olası hatalar) ele alarak modeli eğitime hazırlamak.
# ------------------------------------------------------------------------------

# --- ADIM 1: Kütüphanelerin Yüklenmesi ve Veri Yollarının Tanımlanması ---
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# Veri yollarını tanımlıyoruz
DATA_PATH = os.path.join("..", "data", "raw")
PROCESSED_DATA_PATH = os.path.join("..", "data", "processed")
CSV_PATH = os.path.join(DATA_PATH, "styles.csv")
IMAGES_PATH = os.path.join(DATA_PATH, "images")

# İşlenmiş verileri kaydedeceğimiz klasör yoksa oluştur
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

print("Kütüphaneler yüklendi ve yollar tanımlandı.")


# --- ADIM 2: Verinin Yüklenmesi ve Problemin Basitleştirilmesi ---

# Ham veriyi yüklüyoruz
try:
    df_raw = pd.read_csv(CSV_PATH, on_bad_lines='skip')
    print(f"Başlangıçtaki toplam veri sayısı: {len(df_raw)}")
except FileNotFoundError:
    print(f"HATA: {CSV_PATH} bulunamadı.")
    # Eğer dosya bulunamazsa kodun devam etmesini engelle
    exit()

# En yaygın 10 kategoriyi seçerek problemi basitleştiriyoruz
top_10_categories = df_raw['articleType'].value_counts().head(10).index.tolist()
print(f"\nÜzerinde çalışacağımız en yaygın 10 kategori:\n{top_10_categories}")

# Veri setini sadece bu 10 kategoriyi içerecek şekilde filtreliyoruz
df = df_raw[df_raw['articleType'].isin(top_10_categories)].copy()
print(f"Filtreleme sonrası veri sayısı: {len(df)}")


# --- ADIM 3: Kayıp Resimlerin Temizlenmesi ---

print("\nKayıp resimler kontrol ediliyor...")

# Her bir satır için ilgili resmin diskte var olup olmadığını kontrol edecek bir fonksiyon
def check_image_exists(image_id):
    image_path = os.path.join(IMAGES_PATH, str(image_id) + ".jpg")
    return os.path.exists(image_path)

# Fonksiyonu tüm veri setine uygulayarak 'image_exists' adında yeni bir sütun oluşturuyoruz
df['image_exists'] = df['id'].apply(check_image_exists)

# Sadece resmi var olan satırları tutuyoruz
missing_images_count = len(df[~df['image_exists']])
df = df[df['image_exists']]

print(f"Kontrol tamamlandı. {missing_images_count} adet kayıp resim veriden çıkarıldı.")
print(f"Temizleme sonrası kalan veri sayısı: {len(df)}")


# --- ADIM 4: Veriyi Eğitim ve Doğrulama Setlerine Ayırma ---

print("\nVeri, Eğitim (%80) ve Doğrulama (%20) setlerine ayrılıyor...")

# 'train_test_split' fonksiyonu ile veriyi ayırıyoruz.
# stratify=df['articleType'] parametresi ÇOK ÖNEMLİ: Bu, ayırma işlemi sırasında
# her iki sette de kategori oranlarının korunmasını sağlar. (Dengesizliğe karşı ilk önlem)
train_df, validation_df = train_test_split(
    df,
    test_size=0.2,          # Verinin %20'si doğrulama seti olacak
    random_state=42,        # Sonuçların tekrarlanabilir olması için sabit bir başlangıç noktası
    stratify=df['articleType'] # Kategori oranlarını koru
)

print(f"Eğitim seti boyutu: {len(train_df)}")
print(f"Doğrulama seti boyutu: {len(validation_df)}")


# --- ADIM 5: Temizlenmiş ve Ayrılmış Veriyi Kaydetme ---

# Gelecekteki adımlarda bu işlemleri tekrar yapmamak için temizlenmiş
# veri setlerini 'processed' klasörüne yeni CSV dosyaları olarak kaydediyoruz.
train_save_path = os.path.join(PROCESSED_DATA_PATH, "train_cleaned.csv")
validation_save_path = os.path.join(PROCESSED_DATA_PATH, "validation_cleaned.csv")

train_df.to_csv(train_save_path, index=False)
validation_df.to_csv(validation_save_path, index=False)

print(f"\nİşlem Tamamlandı!")
print(f"Eğitim verisi şuraya kaydedildi: {train_save_path}")
print(f"Doğrulama verisi şuraya kaydedildi: {validation_save_path}")

Kütüphaneler yüklendi ve yollar tanımlandı.
Başlangıçtaki toplam veri sayısı: 44424

Üzerinde çalışacağımız en yaygın 10 kategori:
['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes', 'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses']
Filtreleme sonrası veri sayısı: 25468

Kayıp resimler kontrol ediliyor...
Kontrol tamamlandı. 3 adet kayıp resim veriden çıkarıldı.
Temizleme sonrası kalan veri sayısı: 25465

Veri, Eğitim (%80) ve Doğrulama (%20) setlerine ayrılıyor...
Eğitim seti boyutu: 20372
Doğrulama seti boyutu: 5093

İşlem Tamamlandı!
Eğitim verisi şuraya kaydedildi: ..\data\processed\train_cleaned.csv
Doğrulama verisi şuraya kaydedildi: ..\data\processed\validation_cleaned.csv
